# Week 5-8 - Coding Quizzes

# Week 5 Quiz

In this example, ﻿Z﻿ = the likelihood of a biking accident, ﻿Y﻿ = speed, and ﻿X﻿ = trail difficulty. We assume that ﻿X﻿ decreases ﻿Y﻿ causally because people decrease their speed on difficult trails. In addition, ﻿Y﻿ and ﻿X﻿ both increase ﻿Z﻿ causally because fast biking on difficult trails leads to accidents. Difficulty will be on a scale from 0 to 1, speed in miles per hour, and likelihood of an accident also on a scale from 0 to 1. (Based on the numbers, I'd say these trails are quite challenging!) 

**Code to create data for this question** 
num = 100000 
 
difficulty = np.random.uniform(0, 1, (num,)) 
 
speed = np.maximum(np.random.normal(15, 5, (num, )) - difficulty * 10, 0) 
 
accident = np.minimum(np.maximum(0.03 * speed + 0.4 * difficulty + np.random.normal(0, 0.3, (num,)), 0), 1) 
 
df = pd.DataFrame({'difficulty': difficulty, 'speed': speed, 'accident': accident}) 

**1.Use ﻿X﻿ to predict ﻿Y﻿ many times via regression with different data sets. Use many samples in each prediction. Which is closest to the average coefficient of ﻿X﻿ if you do the experiment enough times?**

In [15]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression

# Number of observations
num = 100000

# X: Trail Difficulty (0 to 1)
difficulty = np.random.uniform(0, 1, num)

# Y: Speed in mph (decreases as difficulty increases)
speed = np.maximum(np.random.normal(15, 5, num) - difficulty * 10, 0)

# Z: Likelihood of accident (increases with both speed and difficulty)
accident = np.minimum(
    np.maximum(0.03 * speed + 0.4 * difficulty + np.random.normal(0, 0.3, num), 0),
    1
)

# Create DataFrame
df = pd.DataFrame({
    'difficulty': difficulty,
    'speed': speed,
    'accident': accident
})

df.head()

,difficulty,speed,accident
0,0.540915,7.978698,0.413691
1,0.468406,12.956855,0.421508
2,0.827738,4.361888,0.360301
3,0.867407,10.090098,0.876056
4,0.768694,3.180727,0.449822


In [16]:
# ----------------------------------------
# 1. Generate Full Dataset (as in question)
# ----------------------------------------
num = 100000

difficulty = np.random.uniform(0, 1, num)  # X: trail difficulty
speed = np.maximum(np.random.normal(15, 5, num) - difficulty * 10, 0)  # Y: speed
accident = np.minimum(
    np.maximum(0.03 * speed + 0.4 * difficulty + np.random.normal(0, 0.3, num), 0),
    1
)  # Z: accident likelihood

df = pd.DataFrame({
    'difficulty': difficulty,
    'speed': speed,
    'accident': accident
})

# ----------------------------------------
# 2. Function to generate data (reusable)
# ----------------------------------------
def generate_data(n=100000):
    difficulty = np.random.uniform(0, 1, n)
    speed = np.maximum(np.random.normal(15, 5, n) - difficulty * 10, 0)
    accident = np.minimum(
        np.maximum(0.03 * speed + 0.4 * difficulty + np.random.normal(0, 0.3, n), 0),
        1
    )
    return difficulty, speed, accident

# -------------------------------------------------
# 3. Function to estimate average regression slope
#    (E[Coefficient of difficulty → speed])
# -------------------------------------------------
def average_regression_coefficient(num_experiments=200, sample_size=5000):
    betas = []
    for _ in range(num_experiments):
        diff, speed, _ = generate_data(n=sample_size)
        lr = LinearRegression().fit(diff.reshape(-1, 1), speed)
        betas.append(lr.coef_[0])
    return np.mean(betas), np.std(betas)

# ----------------------------------------
# 4. Run Simulation (Question 1 answer)
# ----------------------------------------
avg_beta, beta_std = average_regression_coefficient(200, 5000)

print(f"Average coefficient of X (difficulty → speed): {avg_beta:.3f}")
print(f"Standard deviation of estimates: {beta_std:.3f}")

Average coefficient of X (difficulty → speed): -9.632
Standard deviation of estimates: 0.237


**2. Then use ﻿X﻿ and ﻿Z﻿ to predict ﻿Y﻿ many times via regression with different datasets. Which of these is closest to the average coefficient of ﻿X﻿? Note: In practice, should we run such a regression? We are controlling for ﻿Z﻿, but ﻿Z﻿ is a collider. That is, ﻿Y﻿ and ﻿X﻿ both cause ﻿Z﻿. Should we control of it or are we better off ignoring ﻿Z﻿? Why or why not?** 


In [17]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression

# Function to generate data
def generate_data(n=100000):
    difficulty = np.random.uniform(0, 1, n)
    speed = np.maximum(np.random.normal(15, 5, n) - difficulty * 10, 0)
    accident = np.minimum(
        np.maximum(0.03 * speed + 0.4 * difficulty + np.random.normal(0, 0.3, n), 0),
        1
    )
    return difficulty, speed, accident

# Function to estimate average coefficient of X in regression: speed ~ difficulty + accident
def average_beta_X_with_collision(num_experiments=200, sample_size=5000):
    betas_X = []
    
    for _ in range(num_experiments):
        diff, speed, accident = generate_data(sample_size)

        # Stack X and Z (difficulty and accident) as predictors
        XZ = np.column_stack([diff, accident])
        
        # Regress speed on both
        lr = LinearRegression().fit(XZ, speed)
        
        # Coefficient of difficulty (X)
        betas_X.append(lr.coef_[0])
    
    return np.mean(betas_X), np.std(betas_X)

# Run the simulation
avg_beta_X, std_beta_X = average_beta_X_with_collision(200, 5000)

# Display results
print(f"Average coefficient of X (difficulty → speed | controlling for Z): {avg_beta_X:.3f}")
print(f"Standard deviation: {std_beta_X:.3f}")

Average coefficient of X (difficulty → speed | controlling for Z): -10.345
Standard deviation: 0.227


# Week 6 Quiz

Assessment Content
Given a dataset, match treated (﻿X equals 1﻿) to untreated (﻿X equals 0﻿) based on the confounder (﻿Z﻿).
Find the average treatment effect (each item corresponds to one counterfactual) where the counterfactual is the nearest item in the other group (you can use ﻿N e a r e s t N e i g h b o r s﻿ for this.)
Find the average treatment effect on the treated, where each treated item corresponds to a counterfactual untreated item, but we otherwise ignore the untreated items.
Find the average treatment effect on the untreated, where each untreated item corresponds to a counterfactual treated item, but we otherwise ignore the treated items.
Find the optimal treatment effect, which is the maximum treatment effect across all untreated items (i.e., it ends up considering only a single untreated item with its single counterfactual). 
Use the file homework_6.1.csv. 

**1.Which is closest to the average treatment effect?**

In [18]:
import pandas as pd
import numpy as np
from sklearn.neighbors import NearestNeighbors
from sklearn.linear_model import LinearRegression

# 1. Load the dataset
url = "https://raw.githubusercontent.com/joshua-vonkorff/DX702-mod-6/main/homework_6.1.csv"
df = pd.read_csv(url)

# 2. Identify key columns
y_col = "Y"
x_col = "X"
z_cols = [c for c in df.columns if c.startswith("Z")]
if not z_cols and "Z" in df.columns:
    z_cols = ["Z"]
assert {x_col, y_col}.issubset(df.columns), f"Dataset missing {x_col} or {y_col}."

# 3. Split treated / untreated groups 
treated   = df[df[x_col] == 1].reset_index(drop=True)
untreated = df[df[x_col] == 0].reset_index(drop=True)

# Feature matrices for matching
Z_t = treated[z_cols].to_numpy()
Z_u = untreated[z_cols].to_numpy()

# 4. Matching nearest neighbor 
# (A) For treated units → match to untreated
nn_u = NearestNeighbors(n_neighbors=1, algorithm="auto").fit(Z_u)
dist_tu, idx_tu = nn_u.kneighbors(Z_t, return_distance=True)
match_u_for_t = untreated.iloc[idx_tu.ravel()].reset_index(drop=True)

# (B) For untreated units → match to treated
nn_t = NearestNeighbors(n_neighbors=1, algorithm="auto").fit(Z_t)
dist_ut, idx_ut = nn_t.kneighbors(Z_u, return_distance=True)
match_t_for_u = treated.iloc[idx_ut.ravel()].reset_index(drop=True)

# 5. Compute treatment effect estimates 
# ATT: treated only
att_diffs = treated[y_col].to_numpy() - match_u_for_t[y_col].to_numpy()
ATT = att_diffs.mean()

# ATU: untreated only
atu_diffs = match_t_for_u[y_col].to_numpy() - untreated[y_col].to_numpy()
ATU = atu_diffs.mean()

# ATE: average across all units (each matched once)
ATE = np.concatenate([att_diffs, atu_diffs]).mean()

# Optimal treatment effect (max effect among untreated units)
optimal_effect_untreated = atu_diffs.max()
opt_idx = atu_diffs.argmax()
opt_example = {
    "untreated_index": int(opt_idx),
    "untreated_Y": float(untreated.iloc[opt_idx][y_col]),
    "matched_treated_Y": float(match_t_for_u.iloc[opt_idx][y_col]),
    "effect": float(optimal_effect_untreated),
}

#  6. Print results 
print(f"ATT (treated only)                    : {ATT:.3f}")
print(f"ATU (untreated only)                  : {ATU:.3f}")
print(f"ATE (each unit matched once)          : {ATE:.3f}")
print(f"Optimal treatment effect among untreated: {optimal_effect_untreated:.3f}")
print("Example untreated unit with max effect:", opt_example)

ATT (treated only)                    : 1.846
ATU (untreated only)                  : 1.549
ATE (each unit matched once)          : 1.695
Optimal treatment effect among untreated: 2.172
Example untreated unit with max effect: {'untreated_index': 152, 'untreated_Y': -1.459379234, 'matched_treated_Y': 0.7130906515770082, 'effect': 2.172469885577008}


**2.Which is closest to the average treatment effect on the treated?**

In [19]:
import pandas as pd
import numpy as np
from sklearn.neighbors import NearestNeighbors

# Load data from GitHub
url = "https://raw.githubusercontent.com/joshua-vonkorff/DX702-mod-6/main/homework_6.1.csv"
df = pd.read_csv(url)

# Columns
y_col = "Y"   # outcome
x_col = "X"   # treatment
z_cols = [c for c in df.columns if c.startswith("Z")]  # confounder(s)

# Split treated and untreated
treated = df[df[x_col] == 1].reset_index(drop=True)
untreated = df[df[x_col] == 0].reset_index(drop=True)

# Extract Z values
Z_treated = treated[z_cols].to_numpy()
Z_untreated = untreated[z_cols].to_numpy()

# Nearest neighbor matching: each treated finds nearest untreated
nn = NearestNeighbors(n_neighbors=1).fit(Z_untreated)
distances, indices = nn.kneighbors(Z_treated)

# Get matched untreated outcomes
matched_untreated = untreated.iloc[indices.flatten()][y_col].to_numpy()

# ATT = Avg(Y_treated - Y_matched_untreated)
ATT = (treated[y_col].to_numpy() - matched_untreated).mean()

print(f"Average Treatment Effect on the Treated (ATT): {ATT:.3f}")

Average Treatment Effect on the Treated (ATT): 1.846


**3. Which is closest to the average treatment effect on the untreated?**

In [20]:
import pandas as pd
import numpy as np
from sklearn.neighbors import NearestNeighbors

# Load data
url = "https://raw.githubusercontent.com/joshua-vonkorff/DX702-mod-6/main/homework_6.1.csv"
df = pd.read_csv(url)

# Columns
y_col = "Y"   # outcome
x_col = "X"   # treatment
z_cols = [c for c in df.columns if c.startswith("Z")]  # confounders

# Split treated and untreated groups
treated = df[df[x_col] == 1].reset_index(drop=True)
untreated = df[df[x_col] == 0].reset_index(drop=True)

# Extract Z values for matching
Z_treated = treated[z_cols].to_numpy()
Z_untreated = untreated[z_cols].to_numpy()

# Nearest neighbor matching for untreated → find closest treated
nn = NearestNeighbors(n_neighbors=1).fit(Z_treated)
distances, indices = nn.kneighbors(Z_untreated)

# Get matched treated outcomes
matched_treated = treated.iloc[indices.flatten()][y_col].to_numpy()

# ATU = average of [Y(1) - Y(0) | X=0]
atu = (matched_treated - untreated[y_col].to_numpy()).mean()

print(f"Average Treatment Effect on the Untreated (ATU): {atu:.3f}")

Average Treatment Effect on the Untreated (ATU): 1.549


**4.Which is closest to the optimal treatment effect?**

In [21]:
import pandas as pd
import numpy as np
from sklearn.neighbors import NearestNeighbors

# Load dataset from GitHub
url = "https://raw.githubusercontent.com/joshua-vonkorff/DX702-mod-6/main/homework_6.1.csv"
df = pd.read_csv(url)

# Identify columns
y_col = "Y"
x_col = "X"
z_cols = [c for c in df.columns if c.startswith("Z")]

# Split groups
treated = df[df[x_col] == 1].reset_index(drop=True)
untreated = df[df[x_col] == 0].reset_index(drop=True)

# Use Z values for matching
Z_treated = treated[z_cols].to_numpy()
Z_untreated = untreated[z_cols].to_numpy()

# For each untreated, find nearest treated
nn = NearestNeighbors(n_neighbors=1).fit(Z_treated)
distances, indices = nn.kneighbors(Z_untreated)

# Matched treated outcomes
matched_treated = treated.iloc[indices.flatten()][y_col].to_numpy()
untreated_outcomes = untreated[y_col].to_numpy()

# Treatment effect per untreated = Y(1) - Y(0)
effects_untreated = matched_treated - untreated_outcomes

# Optimal treatment effect (maximum individual effect)
optimal_treatment_effect = effects_untreated.max()

print(f"Optimal Treatment Effect: {optimal_treatment_effect:.3f}")

Optimal Treatment Effect: 2.172


# Week 7 Quiz

Suppose that a process can be modeled via linear regression: 

W = np.random.normal(0, 1, (1000,))
X = W + np.random.normal(0, 1, (1000,)) Z = np.random.normal(0, 1, (1000,)) 
Y = X + Z + W + np.random.normal(0, 1, (1000,))

**1. Which is closest to the correlation of ﻿X﻿ with the error term in the equation for ﻿Y﻿?**

In [22]:
import numpy as np
import pandas as pd

np.random.seed(0)

# Generate data
W = np.random.normal(0, 1, 1000)
X = W + np.random.normal(0, 1, 1000)
Z = np.random.normal(0, 1, 1000)
noise = np.random.normal(0, 1, 1000)

# True model
Y = X + Z + W + noise

# Error term (residual from true model)
error = Y - (X + Z + W)

# Correlation between X and the error
corr = np.corrcoef(X, error)[0, 1]
print(f"Correlation between X and error term: {corr:.3f}")

Correlation between X and error term: 0.018


**2.If ﻿Y﻿ is written as depending on ﻿X﻿ and ﻿Z﻿ only, ﻿W﻿ is part of the error term. Which, then, is closest to the expected correlation of ﻿X﻿ with the error term in the equation for ﻿Y﻿?**


In [23]:
import numpy as np

def corr_X_with_error(n=1000, reps=200, seed=0):
    rng = np.random.default_rng(seed)
    cors = []
    for _ in range(reps):
        W   = rng.normal(0, 1, n)
        eta = rng.normal(0, 1, n)   # noise in X
        Z   = rng.normal(0, 1, n)
        eps = rng.normal(0, 1, n)   # structural noise in Y

        X = W + eta
        Y = X + Z + W + eps          # true DGP: Y = X + Z + W + eps

        # If we (incorrectly) model Y ~ X + Z, the error term is u = W + eps
        u = W + eps

        cors.append(np.corrcoef(X, u)[0, 1])
    return float(np.mean(cors)), float(np.std(cors))

avg_corr, sd_corr = corr_X_with_error()
print(f"E[ corr(X, error) ] ≈ {avg_corr:.3f}  (sd ≈ {sd_corr:.3f})")

E[ corr(X, error) ] ≈ 0.498  (sd ≈ 0.023)


**3.In the data frame for homework_7.1.csv, control for W by regressing ﻿Y﻿ on ﻿X﻿ and ﻿Z﻿ at the following constant values of ﻿W﻿: -1, 0, and 1. (You cannot literally use a constant value of ﻿W﻿, of course, or you will have only one data point! How will you manage this?) The question is: Is the coefficient of ﻿X﻿**

In [24]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression

# Load dataset
url = "https://raw.githubusercontent.com/joshua-vonkorff/DX702-mod-6/main/homework_7.1.csv"
df = pd.read_csv(url)

# Function to compute beta_X when W is approximately equal to a chosen value
def beta_X_given_W(df, w_value, bandwidth=0.15):
    subset = df[np.abs(df['W'] - w_value) < bandwidth]
    X_vars = subset[['X', 'Z']]
    y = subset['Y']
    model = LinearRegression().fit(X_vars, y)
    return model.coef_[0], len(subset)   # return coefficient for X and sample size

# Evaluate β_X around W = -1, 0, 1
results = []
for w in [-1, 0, 1]:
    beta_x, n = beta_X_given_W(df, w, bandwidth=0.15)
    results.append((w, beta_x, n))
    print(f"W ≈ {w} → Coefficient of X = {beta_x:.3f}   (samples = {n})")

# Check if coefficient is increasing
if results[0][1] < results[1][1] < results[2][1]:
    print("\n✅ The coefficient of X is increasing as W increases.")
else:
    print("\n❌ The coefficient of X is NOT consistently increasing with W.")

W ≈ -1 → Coefficient of X = 0.870   (samples = 727)
W ≈ 0 → Coefficient of X = 1.381   (samples = 1150)
W ≈ 1 → Coefficient of X = 1.963   (samples = 689)

✅ The coefficient of X is increasing as W increases.


**4. def make_error(corr_const, num):** 

 err = list() 

    prev = np.random.normal(0, 1) 

 for n in range(num): 

    prev = corr_const * prev + (1 - corr_const) * np.random.normal(0, 1) 

    err.append(prev) 

return np.array(err) 



**Create a linear regression model that uses this function as the error for both (a) the treatment, ﻿X﻿, and (b) the outcome, ﻿Y﻿. (You can use random normal error for any other covariates, if you have them.)** 



As corr_const increases from 0.2 to 0.5 to 0.8, find (i) the standard deviation of the estimate of the ﻿X﻿ coefficient over many trials, and (ii) the mean of the standard error estimate of the ﻿X﻿ coefficient over many trials. 



When corr_const increases, then: 



Hint: don't forget to include an intercept in your regression


In [25]:
import numpy as np

# -------------------------------
# Given in the prompt
# -------------------------------
def make_error(corr_const, num):
    err = []
    prev = np.random.normal(0, 1)
    for _ in range(num):
        prev = corr_const * prev + (1 - corr_const) * np.random.normal(0, 1)
        err.append(prev)
    return np.array(err)

# -------------------------------
# OLS with intercept; return beta_X and its classic OLS SE
# -------------------------------
def ols_beta_and_se(X, Y):
    n = len(Y)
    Xmat = np.column_stack([np.ones(n), X])      # intercept + X
    # OLS (normal equations)
    XtX = Xmat.T @ Xmat
    XtX_inv = np.linalg.inv(XtX)
    beta_hat = XtX_inv @ (Xmat.T @ Y)
    # Residual variance and SE for beta_X (index 1)
    resid = Y - Xmat @ beta_hat
    dof = n - Xmat.shape[1]
    sigma2_hat = (resid @ resid) / dof
    var_beta = sigma2_hat * XtX_inv
    se_beta_X = np.sqrt(var_beta[1, 1])
    return beta_hat[1], se_beta_X   # return beta for X and its SE

# -------------------------------
# One Monte Carlo block for a given corr_const
# -------------------------------
def run_block(corr_const, n=1000, reps=500, seed=0):
    rng = np.random.default_rng(seed)
    betas = []
    ses   = []
    for r in range(reps):
        # Errors for X and Y have the SAME autocorrelation structure (independent draws)
        ex = make_error(corr_const, n)
        ey = make_error(corr_const, n)
        # Treatment and outcome
        X = 1.0 + ex                              # centered-ish regressor with AR(1)-like noise
        Y = 2.0 + 1.5 * X + ey                    # true beta_X = 1.5 (intercept included)
        bX, seX = ols_beta_and_se(X, Y)
        betas.append(bX)
        ses.append(seX)
    betas = np.array(betas)
    ses   = np.array(ses)
    sd_beta = betas.std(ddof=1)                   # (i) SD of beta-hat across trials
    mean_se = ses.mean()                          # (ii) mean of OLS SE across trials
    ratio   = sd_beta / mean_se                   # should increase with corr_const
    return sd_beta, mean_se, ratio

# -------------------------------
# Run for corr_const = 0.2, 0.5, 0.8
# -------------------------------
for c in [0.2, 0.5, 0.8]:
    sd_b, mean_se, ratio = run_block(c, n=1000, reps=600, seed=42)
    print(f"corr_const = {c:.1f}  ->  SD(beta_X) = {sd_b:.4f},  mean(SE) = {mean_se:.4f},  ratio = {ratio:.3f}")

corr_const = 0.2  ->  SD(beta_X) = 0.0315,  mean(SE) = 0.0317,  ratio = 0.996
corr_const = 0.5  ->  SD(beta_X) = 0.0410,  mean(SE) = 0.0316,  ratio = 1.294
corr_const = 0.8  ->  SD(beta_X) = 0.0667,  mean(SE) = 0.0316,  ratio = 2.111


# Week 8 Quiz

**Using homework_8.1.csv, find the Average treatment effect with inverse probability weighting. Then, include your code and a written explanation of your work (mentioning any choices or strategies you made in writing the code) in your homework reflection.**  

**Question 1 and 2**
1. the ATE is closest to: 
2. The propensity scores of the first three items in the dataframe are closest to: 

In [26]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression

# --- load data ---
url = "https://raw.githubusercontent.com/joshua-vonkorff/DX702-mod-6/refs/heads/main/homework_8.1.csv"
df = pd.read_csv(url)

# Column names used by the assignment
T_COL = "X"   # treatment (0/1)
Y_COL = "Y"   # outcome (numeric)
# Use every other column as Z (confounders) that predict treatment
Z_COLS = [c for c in df.columns if c not in (T_COL, Y_COL)]

# One-hot encode any categoricals; leave numbers as-is
Z = pd.get_dummies(df[Z_COLS], drop_first=True)

# --- 1) Fit propensity model: P(X=1 | Z) ---
logit = LogisticRegression(max_iter=2000, solver="lbfgs")
logit.fit(Z, df[T_COL])
ps = pd.Series(logit.predict_proba(Z)[:, 1], index=df.index, name="propensity")

# Show the first three propensity scores (rounded so you can match the MCQ)
print("First three propensity scores:", np.round(ps.iloc[:3].to_numpy(), 2).tolist())

# --- 2) Compute IPW ATE ---
# Weights: 1/p for treated, 1/(1-p) for control (clipped for numerical stability)
p = np.clip(ps.values, 1e-6, 1 - 1e-6)
treated = df[T_COL].values.astype(int)
w = np.where(treated == 1, 1.0 / p, 1.0 / (1.0 - p))

# Weighted means by treatment arm
y = df[Y_COL].values.astype(float)
w1 = w[treated == 1]
y1 = y[treated == 1]
w0 = w[treated == 0]
y0 = y[treated == 0]

def wmean(values, weights):
    return np.sum(weights * values) / np.sum(weights)

mu1 = wmean(y1, w1)
mu0 = wmean(y0, w0)
ate_ipw = mu1 - mu0
print(f"IPW ATE estimate: {ate_ipw:.3f}")

# stabilized weights (sometimes numerically nicer)
p_treat = treated.mean()
sw = np.where(treated == 1, p_treat / p, (1 - p_treat) / (1 - p))
mu1_sw = wmean(y1, sw[treated == 1])
mu0_sw = wmean(y0, sw[treated == 0])
ate_ipw_sw = mu1_sw - mu0_sw
print(f"IPW ATE (stabilized): {ate_ipw_sw:.3f}")

First three propensity scores: [0.81, 0.53, 0.66]
IPW ATE estimate: 2.271
IPW ATE (stabilized): 2.271


**Using homework_8.2.csv, match all treated items to the single nearest untreated item using the Mahalanobis distance. (Do this with replacement — the same untreated item can be used again.)** 


* Use the Mahalanobis function from scipy.spatial.distance 

* For the inverse covariance matrix, use all ﻿Z 1﻿ values and all ﻿Z 2﻿ values, make them into a ﻿2 x N﻿ matrix, find its ﻿2 x 2﻿ covariance, and invert. 

**Question 3 and 4**

3.the ATE is closest to: 

4. Find the nearest Z1 and Z2 values of the treated item with the least common support (the farthest Mahalanobis distance from the untreated).  

In [27]:
import numpy as np
import pandas as pd

try:
    from scipy.spatial.distance import mahalanobis
    SCIPY_OK = True
except Exception:
    SCIPY_OK = False

# 1) Load data
url = "https://raw.githubusercontent.com/joshua-vonkorff/DX702-mod-6/refs/heads/main/homework_8.2.csv"
df = pd.read_csv(url)

# Expect columns: X (treatment 0/1), Y (outcome), Z1, Z2
assert {"X","Y","Z1","Z2"}.issubset(df.columns), "Expected columns X, Y, Z1, Z2"

Z_cols = ["Z1", "Z2"]
Z = df[Z_cols].to_numpy()
T = df["X"].to_numpy().astype(int)
Y = df["Y"].to_numpy().astype(float)

# 2) Build inverse covariance of Z (using all rows as instructed)
#    cov is 2x2, invert it
cov = np.cov(Z.T, bias=False)          # shape (2,2)
cov_inv = np.linalg.inv(cov)

# Helper to compute Mahalanobis distances between one z and many z0 rows
def maha_to_many(z_row, Z_many, VI):
    # d^2 = (z - z0)^T VI (z - z0)
    diffs = Z_many - z_row
    return np.sqrt(np.sum(diffs @ VI * diffs, axis=1))

# Split indices
idx_treated = np.where(T == 1)[0]
idx_control = np.where(T == 0)[0]

Z_t = Z[idx_treated]
Z_c = Z[idx_control]
Y_t = Y[idx_treated]
Y_c = Y[idx_control]

# 3) For each treated, find nearest untreated by Mahalanobis (with replacement)
nearest_control_idx = []
nearest_control_dist = []

for z in Z_t:
    dists = maha_to_many(z, Z_c, cov_inv)
    j = np.argmin(dists)
    nearest_control_idx.append(idx_control[j])
    nearest_control_dist.append(dists[j])

nearest_control_idx = np.array(nearest_control_idx)
nearest_control_dist = np.array(nearest_control_dist)

# 4) Compute ATE = mean(Y_treated - Y_matched_control)
Y_matched_c = Y[nearest_control_idx]
ate = float(np.mean(Y_t - Y_matched_c))
print(f"Mahalanobis NN (with replacement) ATE: {ate:.3f}")

# 5) Least common support: treated with LARGEST nearest distance
worst_treated_pos = int(np.argmax(nearest_control_dist))          # position within treated array
worst_treated_idx = int(idx_treated[worst_treated_pos])           # index in original df
worst_z1, worst_z2 = Z[worst_treated_idx]

print("Treated item with least common support (farthest nearest MD):")
print(f"  index: {worst_treated_idx}")
print(f"  Z1, Z2: ({worst_z1:.1f}, {worst_z2:.1f})")
print(f"  nearest MD distance: {nearest_control_dist[worst_treated_pos]:.3f}")


Mahalanobis NN (with replacement) ATE: 3.438
Treated item with least common support (farthest nearest MD):
  index: 494
  Z1, Z2: (2.7, 0.5)
  nearest MD distance: 1.383
